### https://riptutorial.com/julia-lang/example/24364/quotenode--meta-quot--and-expr--quote-

In [1]:
Meta.parse("$x")

UndefVarError: UndefVarError: `x` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [2]:
x=1
Meta.show_sexpr(:($x + $x))
println("\n")
dump(:($x + $x))

(:call, :+, 1, 1)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1


In [4]:
Meta.show_sexpr(:(a + b))
println("\n")
dump(:(a + b))

(:call, :+, :a, :b)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Symbol b


In [5]:
eval(

quote
$(
    Expr(:quote, 
         :( # ex = :( x = 1; :($x + $x) )
            $(Expr(:$, :x)) + $(Expr(:$, :x))
          ) 
          # $x + $x被最外层:()包括 不然就被求值了,
          # 也因为 Expr(:quote, ex) 中, ex需要原本的表达式
          # 所以 $x + $x就是表达式 需要这对黄色括号
        )
 )
end

)
# 和之前显示的结果相同如下: 
#  $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))

:(1 + 1)

In [7]:
ex = :( x = 1; :($x + $x) )

quote
    x = 1
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W5sZmlsZQ==.jl:1 =#
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end

When evaluated, this expression will evaluate `1` and assign it to `x`, then construct an expression of the form `_ + _` where the `_` will be replaced by the value of `x`. Thus, the result of this should be the expression `1 + 1` (which is not yet evaluated, and so distinct from the value `2`). Indeed, this is the case:<br>

当求值时，该表达式将求值 `1` ，并将其赋值为 `x` ，然后构造形式为 `_ + _` 的表达式，其中 `_` 将被 `x` 的值所取代。因此，结果应该是表达式 `1 + 1` （尚未求值，因此与值 `2` 不同）。事实的确如此：

In [8]:
eval(ex)

:(1 + 1)

Let's say now that we're writing a macro to build these kinds of expressions. Our macro will take an argument, which will replace the `1` in the `ex` above. This argument can be any expression, of course. Here is something that is not quite what we want:<br><br>
假设现在我们正在编写一个宏来构建这类表达式。我们的宏将接受一个参数，它将替换上面的 `ex` 中的 `1` 。当然，这个参数可以是任何表达式。这不是我们想要的东西：

In [9]:
makeex_test1 = 
quote
    :( x = 1+2; :(1 + 1) )
end
Base.remove_linenums!(makeex_test1)

quote
    $(Expr(:quote, quote
    x = 1 + 2
    $(Expr(:quote, :(1 + 1)))
end))
end

In [10]:
eval(makeex_test1) |> display
eval(eval(makeex_test1)) |> display
eval(eval(eval(makeex_test1))) |> display

quote
    x = 1 + 2
    $(Expr(:quote, :(1 + 1)))
end

:(1 + 1)

2

In [11]:
makeex_result =
quote
    x = $Expr(:escape, 1)
    $(Expr(:quote,
           :($(Expr(:$, :x)) + $(Expr(:$, :x)))
          )
     )
end
Base.remove_linenums!(makeex_result)

quote
    x = (Expr)(:escape, 1)
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end

In [14]:
macro makeex(arg)
    show(arg)
    quote
        :( x = $(esc($arg)); :($x + $x) )
    end
end

@makeex (macro with 1 method)

In [13]:
@macroexpand @makeex 1

1

quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X15sZmlsZQ==.jl:4 =#
    Core._expr(:block, Core._expr(:(=), :x, Main.esc(1)), $(QuoteNode(:(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X15sZmlsZQ==.jl:4 =#))), Core._expr(:quote, Core._expr(:call, :+, Core._expr(:$, :x), Core._expr(:$, :x))))
end

In [15]:
Base.remove_linenums!(@macroexpand @makeex 1)

1

quote
    Core._expr(:block, Core._expr(:(=), :x, Main.esc(1)), $(QuoteNode(:(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X15sZmlsZQ==.jl:4 =#))), Core._expr(:quote, Core._expr(:call, :+, Core._expr(:$, :x), Core._expr(:$, :x))))
end

In [16]:
Core._expr(:block, 

Core._expr(:(=), :x, Main.esc(1)), 

# $(
#     QuoteNode(
#         :(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:4 =#)
#     )
#  ), 

Core._expr(:quote, 
            Core._expr(:call, 
                       :+, 
                       Core._expr(:$, :x), 
                       Core._expr(:$, :x))
          )
)

quote
    x = $(Expr(:escape, 1))
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end

In [17]:
Core._expr(:call, :+, 1, 2) == Expr(:call, :+, 1, 2)
# 两者等价,但拼接插入到引用中时, Expr需要 $Expr, Core._expr不需要$ 

true

In [18]:
makeex_r = @makeex 1
Base.remove_linenums!(makeex_r)

1

quote
    x = $(Expr(:escape, 1))
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end

In [19]:
eval(makeex_r)

ErrorException: syntax: invalid syntax (escape 1)

In [22]:
makeex_r2 = @makeex 1 + 1
Base.remove_linenums!(makeex_r2)

:(1 + 1)

quote
    x = $(Expr(:escape, 2))
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end

In [24]:
eval(makeex_r2)

ErrorException: syntax: invalid syntax (escape 2)

The second case is incorrect, because we ought to keep `1 + 1` unevaluated. We fix that by quoting the argument with `Meta.quot`:<br>
第二种情况是不正确的，因为我们应该保持 `1 + 1` 未求值。我们通过引用 `Meta.quot` 来修复这个问题：

In [25]:
Meta.quot(:(1+1)) |> display

eval(Meta.quot(:(1+1))) |> display

eval(eval(Meta.quot(:(1+1)))) |> display

:($(Expr(:quote, :(1 + 1))))

:(1 + 1)

2

In [26]:
Core._expr(:quote, :(1+1)) |> display

Meta.quot(:(1+1)) == Expr(:quote, :(1 + 1)) == :(:(1+1)) == Core._expr(:quote, :(1+1))

:($(Expr(:quote, :(1 + 1))))

true

In [27]:
# :($(Expr(:quote, :(1 + 1)))) # 外括号 相当于 quote end
Base.remove_linenums!(
quote
  :(1+1)  # 拼接插入到引用中时, 要用 $取值 去掉1个冒号
end
)  |> display

# 类似的插入拼接
name = "Wang"
"Hello,$name,how are you" |> display #拼接插入到引号中时, 会去掉 Wang 的冒号

quote
    $(Expr(:quote, :(1 + 1)))
end

"Hello,Wang,how are you"

In [34]:
(:(:(1+1)) == quote :(1+1) end) |> display

Meta.quot(:(1+1)) ==
quote
    $(Expr(:quote, :(1 + 1)))
end # 实质是相等的, 两种表达方式不一样, eval看结果能看出来

false

false

In [28]:
eval(
quote
    :(1+1)
end
)

:(1 + 1)

In [29]:
:( x = $$(Meta.quot(:(1+1))); :($x + $x) )

ErrorException: syntax: "$" expression outside quote around e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X36sZmlsZQ==.jl:1

In [30]:
Base.remove_linenums!(
quote
:( x = $$(Meta.quot(:(1+1))); :($x + $x) )
# Meta.quot(:(1+1)) 把接收的 1+1 表达式:(1+1) 当成整体quot处理
# quot处理后,外加了:(),用$去掉,得到正确的表达式，然后嵌入到大括号 :()中时,还要$拼接

# $$(Meta.quot()) 这个语法就是原封不动地将表达式以字面形式嵌入到引用quote中
# $$$(Meta.quot()) 这个语法就是 表达式求值后 嵌入引用

end
)

quote
    $(Expr(:quote, quote
    x = $(Expr(:$, :($(Expr(:quote, :(1 + 1))))))
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end))
end

In [31]:
Base.remove_linenums!(
quote
:( x = $$(Meta.quot(:(1+1))); :($x + $x) )
# Meta.quot(:(1+1)) 把接收的 1+1 表达式:(1+1) 当成整体处理，quot处理
# quot处理后,外加了:(),用$去掉,得到正确的表达式，然后嵌入到大括号 :()中时,还要$拼接
end
) ==

Base.remove_linenums!(
quote
:( x = $$(Expr(:quote,:(1+1))); :($x + $x) )
# Meta.quot(:(1+1)) 与 Expr(:quote,:(1+1))等价
end
)

true

In [32]:
makeex2_result =
quote
    x = $Expr(:escape, 1)
    $(Expr(:quote,
           :($(Expr(:$, :x)) + $(Expr(:$, :x)))
          )
     )
end
Base.remove_linenums!(makeex_result)

quote
    x = (Expr)(:escape, 1)
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end

In [33]:
macro makeex2(arg)
    @show arg    
    @show :($arg)
    println("\n")

    @show Meta.quot(arg)
    @show eval(eval(Meta.quot(arg)))
    println("\n")

    @show :(arg)
    @show :($arg) == arg
    @show Meta.quot(arg)== arg

    quote
        :( x = $$(Meta.quot(arg)); :($x + $x) )
    end
end

@makeex2 (macro with 1 method)

In [34]:
@makeex2 1 + 1


arg = :(1 + 1)
$(Expr(:quote, :($(Expr(:$, :arg))))) = :(1 + 1)


Meta.quot(arg) = :($(Expr(:quote, :(1 + 1))))
eval(eval(Meta.quot(arg))) = 2


:arg = :arg
$(Expr(:quote, :($(Expr(:$, :arg))))) == arg = true
Meta.quot(arg) == arg = false


quote
    x = 1 + 1
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X43sZmlsZQ==.jl:15 =#
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end

```julia
@show :($arg) #为什么是：
$(Expr(:quote, :($(Expr(:$, :arg)))))
```
+ `arg` 接收时是表达式，加有冒号`:()`. `:($arg)` 的括号是 `Expr(:quote,ex)`, `ex` 表达式 `<$arg>`, 即 `Expr(:$, :arg)`, `Expr` 得 `$`取值后 才能拼接插入其他引用中，`Expr(:quote,ex)` 的 `ex` 又得是表达式的本来面目, `$`取值 去掉的冒号得加用`:()`加回来.<br>

+ **`@show` 显示的是 插入到其他引用的形式.** 本来是 `:(:(1+1))` 经过最左的 `$` 成了 `:(1+1)`, 和 `arg :(1+1)` 的本质不同。<br>

+ **`@show` Meta.quot 或 Expr(:quote, arg)时, 是原生引用形式 (带外括号)**，不是插入到其他引用时所需的形式(去掉外括号)<br>

`Meta.quot(arg) == :(:(1+1))`, 所以 **`x = $$(Meta.quot(arg))` 两次`$` 才是 arg的字面意思**

In [51]:
arg = :(1+1)
Meta.quot(arg) |> display
Expr(:quote, arg) |> display

@show Meta.quot(arg)
@show Expr(:quote, arg) # 与Meta.quot(arg)本来就等价

:($(Expr(:quote, :(1 + 1))))

:($(Expr(:quote, :(1 + 1))))

Meta.quot(arg) = :($(Expr(:quote, :(1 + 1))))
Expr(:quote, arg) = :($(Expr(:quote, :(1 + 1))))


:($(Expr(:quote, :(1 + 1))))

In [38]:
Meta.quot(arg) == :(:(1+1))

true

In [37]:
:( x = 1 + 1; :(2 + 2) )

quote
    x = 1 + 1
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X51sZmlsZQ==.jl:1 =#
    $(Expr(:quote, :(2 + 2)))
end

In [39]:
eval(:( x = 1 + 1; :(2 + 2) )) |> display
eval(eval(:( x = 1 + 1; :(2 + 2) ))) |> display

:(2 + 2)

4

要生成的
```julia
:( x = $$(Meta.quot(arg)); :($x + $x) )
```
为什么是
```julia
quote
    x = 1 + 1    
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end
```

In [40]:
$(  # Expr都会伴随 $
    Expr(:quote, # 碰到:(用Expr(:quote, ex), ex需要时原生表达式的样子 所以加蓝括号， 不然加号+会执行
         :(
            $(Expr(:$, :x)) + $(Expr(:$, :x))
          )
        )
)

ErrorException: syntax: "$" expression outside quote around e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X54sZmlsZQ==.jl:1

In [41]:
# :( x = 1 + 1; :(2 + 2) )

eval(

Expr(:quote, # 碰到:(用Expr(:quote, ex), ex需要时原生表达式的样子 所以加蓝括号
         :(
            2 + 2
          )
    )

)

:(2 + 2)

Macro hygiene does not apply to the contents of a quote, so escaping is not necessary in this case (and in fact not legal) in this case.<br>
宏卫生不适用于引用的内容，因此在这种情况下不需要转义（实际上也是不合法的）。<br>

As mentioned earlier, `Meta.quot` allows interpolation. So let's try that out:<br>
如前所述， `Meta.quot` 允许插值。我们来试一下：

In [42]:
@makeex2 1 + $(sin(1))

arg = :(1 + $(Expr(:$, :(sin(1)))))
$(Expr(:quote, :($(Expr(:$, :arg))))) = :(1 + $(Expr(:$, :(sin(1)))))


Meta.quot(arg) = :($(Expr(:quote, :(1 + $(Expr(:$, :(sin(1))))))))
eval(eval(Meta.quot(arg))) = 1.8414709848078965


:arg = :arg
$(Expr(:quote, :($(Expr(:$, :arg))))) == arg = true
Meta.quot(arg) == arg = false


quote
    x = 1 + 0.8414709848078965
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X43sZmlsZQ==.jl:15 =#
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end

In [43]:
let az = 0.5
    @makeex2 1 + $az # 为什么没有定义
end

arg = :(1 + $(Expr(:$, :az)))
$(Expr(:quote, :($(Expr(:$, :arg))))) = :(1 + $(Expr(:$, :az)))


Meta.quot(arg) = :($(Expr(:quote, :(1 + $(Expr(:$, :az))))))


LoadError: LoadError: UndefVarError: `az` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X61sZmlsZQ==.jl:2

In [45]:
q = 0.5
@makeex2 1 + $q

arg = :(1 + $(Expr(:$, :q)))
$(Expr(:quote, :($(Expr(:$, :arg))))) = :(1 + $(Expr(:$, :q)))


Meta.quot(arg) = :($(Expr(:quote, :(1 + $(Expr(:$, :q))))))
eval(eval(Meta.quot(arg))) = 1.5


:arg = :arg
$(Expr(:quote, :($(Expr(:$, :arg))))) == arg = true
Meta.quot(arg) == arg = false


quote
    x = 1 + 0.5
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X43sZmlsZQ==.jl:15 =#
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end

From the first example, we see that interpolation allows us to inline the `sin(1)`, instead of having the expression be a literal `sin(1)`. The second example shows that this interpolation is done in the macro invocation scope, not the macro's own scope. That's because our macro hasn't actually evaluated any code; all it's doing is generating code. The evaluation of the code (which makes its way into the expression) is done when the expression the macro generates is actually run.<br>

从第一个例子中，我们看到插值允许我们内联 `sin(1)` ，而不是让表达式是文字 `sin(1)` 。第二个例子表明，这种插值是在宏调用作用域中完成的，而不是在宏自己的作用域中。这是因为我们的**宏实际上并没有计算任何代码；它所做的就是生成代码**。当宏生成的表达式实际运行时，代码的求值（进入表达式）就完成了。<br>

What if we had used `QuoteNode` instead? As you may guess, since `QuoteNode` prevents interpolation from happening at all, this means it won't work.<br>

如果我们用 `QuoteNode` 呢？正如您可能猜到的那样，由于 `QuoteNode` 完全阻止了插值的发生，这意味着它将不起作用。

In [52]:
macro makeex3(arg)
    quote
        :( x = $$(QuoteNode(arg)); :($x + $x) )
    end
end

@makeex3 (macro with 1 method)

In [58]:
@makeex3 1 + $(sin(1))

quote
    x = 1 + $(Expr(:$, :(sin(1))))
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X64sZmlsZQ==.jl:3 =#
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end

In [56]:
let by = 0.5
    @makeex3 1 + $by
end

quote
    x = 1 + $(Expr(:$, :by))
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X64sZmlsZQ==.jl:3 =#
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end

In [59]:
eval(@makeex3 $(sin(1)))

ErrorException: syntax: "$" expression outside quote around e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X64sZmlsZQ==.jl:3

In this example, we might agree that `Meta.quot` gives greater flexibility, as it allows interpolation. So why might we ever consider using `QuoteNode`? In some cases, we may not actually desire interpolation, and **actually want the literal `$` expression**. When would that be desirable? Let's consider a generalization of `@makeex` where we can pass additional arguments determining what comes to the left and right of the `+` sign:<br>

在这个例子中，我们可能会同意 `Meta.quot` 提供了更大的灵活性，因为它允许插值。那么为什么我们会考虑使用 `QuoteNode` 呢？在某些情况下，我们可能实际上不需要插值，而实际上需要文字 `$` 表达式。什么时候合适？让我们考虑 `@makeex` 的泛化，我们可以传递额外的参数来决定 `+` 符号的左右：

In [93]:
macro makeex4(expr, left, right)
    @show left
    @show Meta.quot(left)
    @show eval(eval(Meta.quot(left)))

    quote
        quote
            $$(Meta.quot(expr))  # quot多1个冒号 $取值后于院表达式等同, 插入引用时要来个$, 所以两个$
            :(
                $(
                   $(
                      $(Meta.quot(left)) # (1) 与原表达式等同
                    ) # (2) 插入引用时要来个$, or 计算表达式
               ) + # (3) 再对表达式求值, or 插入黄色括号中
               
               $$$(Meta.quot(right))
             )
        end
    end
end

@makeex4 (macro with 1 method)

In [94]:
@makeex4 x=1 1/2 1/4

left = :(1 / 2)
Meta.quot(left) = :($(Expr(:quote, :(1 / 2))))
eval(eval(Meta.quot(left))) = 0.5


quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y102sZmlsZQ==.jl:8 =#
    x = 1
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y102sZmlsZQ==.jl:9 =#
    $(Expr(:quote, :($(Expr(:$, :(1 / 2))) + $(Expr(:$, :(1 / 4))))))
end

In [ ]:
$(
    Expr(:quote, # 圆括号
         :(
            $(Expr(:$, :(1 / 2)))  
            # :(1 / 2) 是通过 $(Meta.quot(left) 得到的
            # Expr(:$, $(Meta.quot(left))) = $$(Meta.quot(left)) # 表达式求值
            # $Expr(:$, $(Meta.quot(left))) = $$$(Meta.quot(left) # 执行Expr表达式 或者 叫 插入蓝色括号中

            + $(Expr(:$, :(1 / 4)))
          )
        )
 )

In [103]:
eval(:(1 / 2)) |> display

Expr(:eval, :(1 / 2)) |>display

eval(Expr(:eval, :(1 / 2))) |> display

Expr(:$, :(1 / 2))

0.5

:($(Expr(:eval, :(1 / 2))))

ErrorException: syntax: invalid syntax (eval (call (outerref /) 1 2))

In [104]:
dump(eval(:(1 + 1 / 2)))

Meta.show_sexpr(eval(:( 1+ 1 / 2)))

Float64 1.5
1.5

In [106]:
dump(eval(Meta.quot(:(1))))

Int64 1


In [109]:
eval(@makeex4 x=1 1/2 1/4)

left = :(1 / 2)
Meta.quot(left) = :($(Expr(:quote, :(1 / 2))))
eval(eval(Meta.quot(left))) = 0.5


:(0.5 + 0.25)

In [110]:
@makeex4 x=1 x x

left = :x
Meta.quot(left) = :(:x)
eval(eval(Meta.quot(left))) = 1


quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y102sZmlsZQ==.jl:8 =#
    x = 1
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y102sZmlsZQ==.jl:9 =#
    $(Expr(:quote, :($(Expr(:$, :x)) + $(Expr(:$, :x)))))
end

In [111]:
eval(@makeex4 x=1 x x)

left = :x
Meta.quot(left) = :(:x)
eval(eval(Meta.quot(left))) = 1


:(1 + 1)

In [113]:
@macroexpand @makeex4 x=1 x x

left = :x
Meta.quot(left) = :(:x)
eval(eval(Meta.quot(left))) = 1


quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y102sZmlsZQ==.jl:7 =#
    Core._expr(:block, $(QuoteNode(:(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y102sZmlsZQ==.jl:8 =#))), $(Expr(:copyast, :($(QuoteNode(:(x = 1)))))), $(QuoteNode(:(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y102sZmlsZQ==.jl:9 =#))), Core._expr(:quote, Core._expr(:call, :+, Core._expr(:$, :x), Core._expr(:$, :x))))
end

In [ ]:
Core._expr(:block, 
    $(QuoteNode(
                :(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y101sZmlsZQ==.jl:4 =#)
               )
     ), 
     
    $(Expr(:copyast, # $$(Meta.quot(expr))的最终效果是输入参数表达式字面相同插入到这, 相当于复制Expr(:copyast, ex)
           :(
             $(QuoteNode(:(x = 1)))
            )
          )
     ), 
    
    $(QuoteNode(
                :(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y101sZmlsZQ==.jl:5 =#)
               )
     ), 
     
     Core._expr(:quote, 
                 Core._expr(:call, 
                            :+, 
                            Core._expr(:$, :x), Core._expr(:$, :x))
               )
)

In [ ]:
$(
Expr(:copyast, # $$(Meta.quot(expr))的最终效果是输入参数表达式字面相同插入到这, 相当于复制Expr(:copyast, ex)
     :(
        $(QuoteNode(:(x = 1))) # (1)相当于 :(x=1)
      )  # (2) 相当于 :(:(x=1))
    )
) # (3) 相当于 :(x=1)

In [115]:
Expr(:copyast, (1+1))

:($(Expr(:copyast, 2)))

In [116]:
eval(

Expr(:copyast, 
           :(
             $(QuoteNode(:(1+1))) # (1)相当于 :(1+1)
            )  # (2) 相当于 :(:(1+1)), 加蓝色括号是保护起来, 否则会计算得2
)

) # 多加了蓝色括号, $(相当于eval)复原

:(1 + 1)

In [118]:
Core._expr(:block,
:(x = 1),
:(1 + 1),
)

# 与quote ... end (已有引号) 不同，
# Core._expr() 括号中的参数不用去引号 因为不是引号拼接 

quote
    x = 1
    1 + 1
end

In [119]:
eval(

Core._expr(:block,
:(x = 1),
:(1 + 1),
)

)

2

A limitation of our implementation of `@makeex4` is that we can't use expressions as either the left and right sides of the expression directly, because they get interpolated. In other words, **the expressions may get evaluated for interpolation, but we might want them preserved**. (Since there are many levels of quoting and evaluation here, let us clarify: our macro generates code that constructs an expression that when evaluated produces another expression. Phew!)<br>

 `@makeex4` 实现的一个限制是，我们不能直接使用表达式作为表达式的左右两边，因为它们会被内插。换句话说，表达式可能会为插值求值，但我们可能希望保留它们。(因为这里有很多层次的引用和求值，让我们澄清一下：我们的**宏生成的代码构造了一个表达式，当求值时会产生另一个表达式**。唷!)

In [121]:
@makeex4 x=1 x/2 x

left = :(x / 2)
Meta.quot(left) = :($(Expr(:quote, :(x / 2))))
eval(eval(Meta.quot(left))) = 0.5


quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y102sZmlsZQ==.jl:8 =#
    x = 1
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y102sZmlsZQ==.jl:9 =#
    $(Expr(:quote, :($(Expr(:$, :(x / 2))) + $(Expr(:$, :x)))))
end

In [122]:
eval(@makeex4 x=1 x/2 x)

left = :(x / 2)
Meta.quot(left) = :($(Expr(:quote, :(x / 2))))
eval(eval(Meta.quot(left))) = 0.5


:(0.5 + 1)

In [123]:
eval(@makeex4 x=1 1/2 1/4)

left = :(1 / 2)
Meta.quot(left) = :($(Expr(:quote, :(1 / 2))))
eval(eval(Meta.quot(left))) = 0.5


:(0.5 + 0.25)

We ought to allow the user to specify when interpolation is to happen, and when it shouldn't. Theoretically, that's an easy fix: we can just remove one of the `$` signs in our application, and let the user contribute their own. What this means is that we interpolate a quoted version of the expression entered by the user (which we've already quoted and interpolated once). This leads to the following code, which can be a little confusing at first, due to the multiple nested levels of quoting and unquoting. Try to read and understand what each escape is for.<br>

我们应该允许用户指定何时进行插值，何时不进行插值。从理论上讲，这是一个简单的解决方案：我们可以在应用程序中删除一个 `$` 标志，并让用户贡献他们自己的标志。这意味着我们插入了用户输入的表达式的引号版本（我们已经引用并插入了一次）。这导致了下面的代码，由于多个嵌套的引用和取消引用级别，一开始可能会让人有点困惑。试着阅读并理解每个逃避的目的。

In [124]:
macro makeex5_pre_test(expr, left, right)
    @show Meta.quot(left)
    # @show Meta.quot($(Meta.quot(left))) # Error "$" expression outside quote

    # Meta.quot(left) 外加1层 如两层冒号, $(Meta.quot(left)) 1层冒号
    quote
        quote
            $$(Meta.quot(expr))
            :(
                $$(Meta.quot(Meta.quot(left))) + $$(Meta.quot(Meta.quot(right)))
             )
        end
    end
end

@makeex5_pre_test (macro with 1 method)

In [125]:
@makeex5_pre_test x=1 1/2 1/4

Meta.quot(left) = :($(Expr(:quote, :(1 / 2))))


UndefVarError: UndefVarError: `left` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [126]:
@macroexpand @makeex5_pre_test x=1 1/2 1/4

Meta.quot(left) = :($(Expr(:quote, :(1 / 2))))


quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y130sZmlsZQ==.jl:7 =#
    Core._expr(:block, $(QuoteNode(:(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y130sZmlsZQ==.jl:8 =#))), $(Expr(:copyast, :($(QuoteNode(:(x = 1)))))), $(QuoteNode(:(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y130sZmlsZQ==.jl:9 =#))), Core._expr(:quote, Core._expr(:call, :+, Core._expr(:$, (Main.Meta).quot((Main.Meta).quot(Main.left))), Core._expr(:$, (Main.Meta).quot((Main.Meta).quot(Main.right))))))
end

In [128]:
Core._expr(:block, 
$(QuoteNode(
            :(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y144sZmlsZQ==.jl:8 =#)
           )
 ), 
 
 $(Expr(:copyast, 
        :(
            $(QuoteNode(:(x = 1)))
         )
       )
  ), 
  
  $(QuoteNode(
              :(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y144sZmlsZQ==.jl:9 =#)
             )
   ), 
   
   Core._expr(:quote, 
              Core._expr(:call, 
                         :+, 

                         Core._expr(:$, 
                                    (Main.Meta).quot( # 这里为什么就是(Main.Meta) 和 (Main.left)
                                                      (Main.Meta).quot(Main.left)
                                                    )
                                   ), 

                         Core._expr(:$, 
                                    (Main.Meta).quot(
                                                      (Main.Meta).quot(Main.right)
                                                    )
                                   )
                        )
             )
)

ErrorException: syntax: "$" expression outside quote around e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y133sZmlsZQ==.jl:1

In [130]:
macro makeex5(expr, left, right)
    @show Meta.quot(left)
    # @show Meta.quot($(Meta.quot(left))) # Error "$" expression outside quote
    
    println("\n")
    @show :(Meta.quot($(Meta.quot(left)))) 

    # Meta.quot(left) 外加1层 如两层冒号, $(Meta.quot(left)) 1层冒号
    quote
        quote
            $$(Meta.quot(expr))
            :(
                $$(Meta.quot($(Meta.quot(left)))) + $$(Meta.quot($(Meta.quot(right))))
             )
        end
    end
end

@makeex5 (macro with 1 method)

In [131]:
@makeex5 x=1 1/2 1/4

Meta.quot(left) = :($(Expr(:quote, :(1 / 2))))


$(Expr(:quote, :(Meta.quot($(Expr(:$, :(Meta.quot(left)))))))) = :(Meta.quot($(Expr(:quote, :(1 / 2)))))


quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y134sZmlsZQ==.jl:11 =#
    x = 1
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y134sZmlsZQ==.jl:12 =#
    $(Expr(:quote, :($(Expr(:$, :($(Expr(:quote, :(1 / 2)))))) + $(Expr(:$, :($(Expr(:quote, :(1 / 4)))))))))
end

In [188]:
Expr(:quote, :(1/2)) |> display
eval(Expr(:quote, :(1/2))) |> display

# 二重quot
again = Expr(:quote, :(1/2)) 
again |> display
"上行为 again" |> display

eval(again) |> display
eval(eval(again)) |> display


:(Meta.quot($(Expr(:quote, :(1 / 2))))) == :(Meta.quot($(Meta.quot(:(1 / 2)))))

:($(Expr(:quote, :(1 / 2))))

:(1 / 2)

:($(Expr(:quote, :(1 / 2))))

"上行为 again"

:(1 / 2)

0.5

true

In [192]:
:(eval(1/2) + eval(1/4))

:(eval(1 / 2) + eval(1 / 4))

In [132]:
eval(@makeex5 x=1 1/2 1/4)  # 为什么 结果不是 (0.5 + 0.25) ???

Meta.quot(left) = :($(Expr(:quote, :(1 / 2))))


$(Expr(:quote, :(Meta.quot($(Expr(:$, :(Meta.quot(left)))))))) = :(Meta.quot($(Expr(:quote, :(1 / 2)))))


:(1 / 2 + 1 / 4)

In [134]:
@macroexpand @makeex5 x=1 1/2 1/4

Meta.quot(left) = :($(Expr(:quote, :(1 / 2))))


$(Expr(:quote, :(Meta.quot($(Expr(:$, :(Meta.quot(left)))))))) = :(Meta.quot($(Expr(:quote, :(1 / 2)))))


quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y134sZmlsZQ==.jl:10 =#
    Core._expr(:block, $(QuoteNode(:(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y134sZmlsZQ==.jl:11 =#))), $(Expr(:copyast, :($(QuoteNode(:(x = 1)))))), $(QuoteNode(:(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y134sZmlsZQ==.jl:12 =#))), Core._expr(:quote, Core._expr(:call, :+, Core._expr(:$, (Main.Meta).quot($(Expr(:copyast, :($(QuoteNode(:(1 / 2)))))))), Core._expr(:$, (Main.Meta).quot($(Expr(:copyast, :($(QuoteNode(:(1 / 4)))))))))))
end

In [135]:
Core._expr(:block, 

# $(QuoteNode(
#             :(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y123sZmlsZQ==.jl:8 =#)
#            )), 

# $$(Meta.quot(expr))是：
$(
    Expr(:copyast,
         :(
            $(QuoteNode(:(x = 1)))
          )
        )
 ), 

# $(QuoteNode(
#             :(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y123sZmlsZQ==.jl:9 =#)
#            )), 


# :(
#    $$(Meta.quot($(Meta.quot(left)))) + $$(Meta.quot($(Meta.quot(right))))
# ) 是：
Core._expr(:quote,
 
           Core._expr(:call, 
                      :+, 


                      Core._expr(:$, 
                                 (Main.Meta).quot(
                                    $(Expr(:copyast, 
                                           :($(QuoteNode(
                                                          :(1 / 2) # 相当于$(Meta.quot(left))
                                                        )
                                              ) # 这步相当于白干
                                            )
                                          )
                                     ) # 复制 
                                 ) # quot
                                ), # 第1次$

                      Core._expr(:$, 
                                 (Main.Meta).quot(
                                     $(Expr(:copyast, 
                                            :($(QuoteNode(:(1 / 4))))
                                           )
                                      )
                                 )
                                )
                     )
         )
)

ErrorException: syntax: "$" expression outside quote around e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y143sZmlsZQ==.jl:1

In [136]:
quote
$(
    Expr(:copyast,
         :(
            $(QuoteNode(
                         # $(Meta.quot(left))
                         Core._expr(:$,
                                    (Main.Meta).quot(:(1 / 2))
                                   )
                       )
             )
          )
        )
 ) 
end ==
 Core._expr(:$, 
                                 (Main.Meta).quot(
                                    $(Expr(:copyast, 
                                           :($(QuoteNode(:(1 / 2))))
                                          )
                                     )
                                 )
                                )

ErrorException: syntax: "$" expression outside quote around e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y144sZmlsZQ==.jl:1

In [143]:
@makeex5 y=1 $y $y  # 这样 宏没有执行 y=1赋值语句, 所以`y` not defined in `Main`

Meta.quot(left) = :($(Expr(:quote, :($(Expr(:$, :y))))))


$(Expr(:quote, :(Meta.quot($(Expr(:$, :(Meta.quot(left)))))))) = :(Meta.quot($(Expr(:quote, :($(Expr(:$, :y)))))))


UndefVarError: UndefVarError: `y` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [144]:
macro makeex55(expr, left, right)
    @show Meta.quot(left)
    # @show Meta.quot($(Meta.quot(left))) # Error "$" expression outside quote

    # Meta.quot(left) 外加1层 如两层冒号, $(Meta.quot(left)) 1层冒号
    quote
        quote
            $$(Meta.quot(expr))
            :(
                $$(Meta.quot(left)) + $$(Meta.quot(right))
             )
        end
    end
end

@makeex55 (macro with 1 method)

In [145]:
eval(@makeex55 x=1 1/2 1/4) 
# @macroexpand @makeex55 x=1 1/2 1/4 展开的表达式里有 Main.left (目前 Main里还不存在)

Meta.quot(left) = :($(Expr(:quote, :(1 / 2))))


UndefVarError: UndefVarError: `left` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [146]:
@macroexpand @makeex55 x=1 1/2 1/4

Meta.quot(left) = :($(Expr(:quote, :(1 / 2))))


quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y146sZmlsZQ==.jl:7 =#
    Core._expr(:block, $(QuoteNode(:(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y146sZmlsZQ==.jl:8 =#))), $(Expr(:copyast, :($(QuoteNode(:(x = 1)))))), $(QuoteNode(:(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y146sZmlsZQ==.jl:9 =#))), Core._expr(:quote, Core._expr(:call, :+, Core._expr(:$, (Main.Meta).quot(Main.left)), Core._expr(:$, (Main.Meta).quot(Main.right)))))
end

Things started well, but something has gone wrong. The macro's generated code is trying to interpolate the copy of `y` in the macro invocation scope; but there is no copy of `y` in the macro invocation scope. Our error is allowing interpolation with the second and third arguments in the macro. To fix this error, we must use `QuoteNode`.<br>

事情开始很顺利，但后来出了问题。宏生成的代码试图在宏调用范围内插入 `y` 的副本；但是在宏调用作用域中没有 `y` 的副本。我们的错误是允许在宏中使用第二个和第三个参数进行插值。要修复这个错误，我们必须使用 `QuoteNode` 。

In [1]:
macro makeex6(expr, left, right)
    quote
        quote
            $$(Meta.quot(expr))
            :($$(Meta.quot($(QuoteNode(left)))) + $$(Meta.quot($(QuoteNode(right)))))
        end
    end
end

@makeex6 (macro with 1 method)

In [2]:
@makeex6 zz=1 1+$zz $zz

quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y155sZmlsZQ==.jl:4 =#
    zz = 1
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y155sZmlsZQ==.jl:5 =#
    $(Expr(:quote, :($(Expr(:$, :($(Expr(:quote, :(1 + $(Expr(:$, :zz)))))))) + $(Expr(:$, :($(Expr(:quote, :($(Expr(:$, :zz)))))))))))
end

In [ ]:
$(Expr(:quote,  # 外层括号
       :($(Expr(:$, 
                :(
                    $(Expr(:quote, 
                           :(1 + $(Expr(:$, :zz)))  # 相当于 $(QuoteNode(left))
                          )
                     ) # 相当于 $(Meta.quot($(QuoteNode(left))))
                 )
               )
          ) # 相当于 $$(Meta.quot($(QuoteNode(left))))
         
         + 

         $(Expr(:$, 
                :(
                    $(Expr(:quote, 
                           :($(Expr(:$, :zz))) # 相当于 $(QuoteNode(right))
                          )
                     ) # 相当于 $(Meta.quot($(QuoteNode(right))))
                 )
               )
          ) # 相当于 $$(Meta.quot($(QuoteNode(right))))

        )
      )
 )

In [4]:
zz # zz 还没有赋值，所以出现未定义错误

UndefVarError: UndefVarError: `zz` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [7]:
eval(@makeex6 zz=1 1+$zz $zz) |> display 
# 宏参数 zz=1 究竟什么时候执行, 得看宏内部是怎么写的

zz |> display  # zz = 1 执行过啦

:((1 + 1) + 1)

1

In [8]:
@makeex6 y=1 1/2 1/4

quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y155sZmlsZQ==.jl:4 =#
    y = 1
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y155sZmlsZQ==.jl:5 =#
    $(Expr(:quote, :($(Expr(:$, :($(Expr(:quote, :(1 / 2)))))) + $(Expr(:$, :($(Expr(:quote, :(1 / 4)))))))))
end

In [9]:
eval(@makeex6 y=1 1/2 1/4)

:(1 / 2 + 1 / 4)

In [10]:
@makeex6 y=1 $y $y

quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y155sZmlsZQ==.jl:4 =#
    y = 1
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y155sZmlsZQ==.jl:5 =#
    $(Expr(:quote, :($(Expr(:$, :($(Expr(:quote, :($(Expr(:$, :y)))))))) + $(Expr(:$, :($(Expr(:quote, :($(Expr(:$, :y)))))))))))
end

In [11]:
eval(@makeex6 y=1 $y $y)

:(1 + 1)

In [19]:
eval(@makeex6 y=1 1+$y/4 $y)

:((1 + 1 / 4) + 1)

By using `QuoteNode`, we have protected our arguments from interpolation. Since `QuoteNode` only has the effect of additional protections, it is never harmful to use `QuoteNode`, unless you desire interpolation. However, understanding the difference makes it possible to understand where and why `Meta.quot` could be a better choice.<br>

通过使用 `QuoteNode` ，我们保护了参数不受插值的影响。因为 `QuoteNode` 只具有额外的保护作用，所以使用 `QuoteNode` 是无害的，除非你想要插值。然而，理解了它们之间的区别，就有可能理解 `Meta.quot` 在哪里以及为什么是更好的选择。<br>

This long exercise is with an example that is plainly too complex to show up in any reasonable application. Therefore, we have justified the following rule of thumb, mentioned earlier:<br>

这个长时间的练习使用的示例显然太复杂，无法在任何合理的应用程序中显示。因此，我们证明了前面提到的以下经验法则是合理的：<br>

+ If you need or want to support interpolation, use `Meta.quot`;<br>
如果您需要或希望支持插值，请使用 `Meta.quot` ；<br><br>
+ If you can't or don't want to allow interpolation, use `QuoteNode`.<br>
如果你不能或不想允许插值，使用 `QuoteNode` 。

### https://discourse.julialang.org/t/quoting-symbols/23308/2

You want QuoteNode: `f(s) = :(someArray[$(QuoteNode(s))])`<br>
**When in doubt, just dump an expression** you would like to create and see what it is made of:

In [2]:
dump(:(someArray[:x]))

Expr
  head: Symbol ref
  args: Array{Any}((2,))
    1: Symbol someArray
    2: QuoteNode
      value: Symbol x


In [5]:
Expr(:ref, :somArray, QuoteNode(:x))

:(somArray[:x])

The `Expr(:quote)` support interpolation expression when evaluated.` QuoteNode` simply return the quoted value. **It’s like the difference between the quote and backquote in lisp**.
这两者并不等同。 `Expr(:quote)` 在计算时支持插值表达式。 `QuoteNode` 只是返回引用的值。这就像lisp中引号和反引号的区别。